In [ ]:
from data_crawler import *
from loguru import logger

def setup_logger(file_name):
    logger.remove()
    logger.add(f"./logs/{file_name}-debug.log", format="{time} {level} {message}", level="DEBUG", enqueue=True)
    logger.add(f"./logs/{file_name}-info.log", format="{time} {level} {message}", level="INFO", enqueue=True, backtrace=True)
    logger.add(f"./logs/{file_name}-error.log", format="{time} {level} {message}", level="ERROR", enqueue=True, backtrace=True, diagnose=True)

setup_logger("data_crawler")

dm = DataModel()
dm.delete_pystore_item()

craw2df = Crawl2DF(4)
craw2df.run()

# new indexing model

In [48]:
from data_provider_new import *

nametmp = "1380-10-10"
nametmp1 = "1390-10-11"
def do(nametmp):
    dftmp = pd.read_excel(f'../excels/1380-10-10.xlsx', header=[0], skiprows=[0,1], convert_float=False)
    dftmp.columns = DataModel.HEADER
    year, month, day = nametmp.split("-")
    date = JalaliDate(year, month, day).todate()
    yearlist = np.full(len(dftmp), year).tolist()
    monthlist = np.full(len(dftmp), month).tolist()
    daylist = np.full(len(dftmp), day).tolist()
    datelist = np.full(len(dftmp), date).tolist()
    dftmp["year"] = yearlist
    dftmp["month"] = monthlist
    dftmp["day"] = daylist
    dftmp["date"] = datelist
    dftmp["date"] = pd.to_datetime(dftmp["date"])
    dftmp["symbol"] = dftmp.symbol.apply(str)
    dftmp["name"] = dftmp["name"].apply(str)
    dftmp = dftmp.astype({"year": int, "month": int, "day": int})
    dftmp = dftmp.set_index(['symbol', 'date'])
    return dftmp
df1 = do(nametmp)
df2 = do(nametmp1)
# dm_new = DataModel()
# dm_new.read_from_df(pd.concat(all_dfs).sort_index())
# self.dm.df = self.dm.df.append(dm_new.df)
# all_dfs = []
# logger.info(f'storing in pystore')
# self.dm.update_df_extensions()
# self.dm.store_in_pystore()
# logger.info(f'stored in pystore')
# self.dm.save_to_csvs(self.excel_location, "master", 1000000)
# logger.info(f'stored in file')
df = pd.concat([df1, df2]).sort_index()
df = df.loc[~df.index.duplicated(keep='last')]
dftmp = df.loc[df.index.get_level_values("symbol")[2]].copy()
